In [1]:
from twikit import Client
import time
import json
from twikit import TwitterException 

USERNAME = 'EloraghEspie'
EMAIL = 'eaespie@gmail.com'
PASSWORD = 'qtr4pnj3tvc_KDU5cax'

# Initialize client
client = Client(language = 'en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

{'flow_token': 'g;171355428707212065:-1713554287139:hlPyIEf6yJdEWxRvu2yf8KpX:14',
 'status': 'success',
 'subtasks': [{'subtask_id': 'LoginSuccessSubtask',
   'open_account': {'user': {'id': 1547081484695216130,
     'id_str': '1547081484695216130',
     'name': 'Eloragh Espie',
     'screen_name': 'EloraghEspie'},
    'next_link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'SuccessExit'},
    'attribution_event': 'login'}},
  {'subtask_id': 'SuccessExit',
   'open_link': {'link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'LoginOpenHomeTimeline'}}},
  {'subtask_id': 'LoginOpenHomeTimeline',
   'open_home_timeline': {'next_link': {'link_type': 'abort',
     'link_id': 'next_link'}}}]}

In [2]:
client.get_cookies()
client.save_cookies('cookies.json')
with open('cookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [42]:
def get_all_tweets(handle, since, until):
    client.load_cookies('cookies.json')
    # we need to scrape the tweets of each user from the period of their election data

    mass_tweets = []
    user_id = client.get_user_by_screen_name(handle)
    all_tweets = []

    tweets = client.search_tweet(
        f'from:{handle} since:{since} until:{until}', 'Latest', count=40
    )

    if len(tweets) == 0:
        print('No tweets available')
        return(handle, user_id, [])
    else:
        while len(all_tweets) >= 0 and len(mass_tweets) < 100:
            tweets1 = [tweet.id for tweet in tweets]
            mass_tweets += tweets1
            time.sleep(1)  # cooldown
            tweets = tweets.next()
            print(len(mass_tweets))

    all_tweets+=mass_tweets

    return(handle, user_id, all_tweets)

In [ ]:
def get_all_tweets2(user_id, since, until):
    # we need to scrape the tweets of each user from the period of their election data
    client.load_cookies('cookies.json')
    all_tweets = []
    for handle in user_id:
        id = client.get_user_by_screen_name(handle)
        tweets = client.search_tweet(
            f'from:{id}', product='Latest', count=20
        )
        print(tweets)

    # this scraper only provides 20 tweets at at time
    # this while loop allows the program to wait ten seconds then seek more tweets
    # this loop will continue until all tweets within the datetime range are exhausted
    '''
    i = 0
    while len(tweets) > 0:
        print(i)
        all_tweets += tweets
        time.sleep(1)  # cooldown
        tweets = tweets.next()
        i+=1
        print(len(all_tweets))
    '''
    print('done')
    return(all_tweets)

In [4]:
def process_tweets(handle, user_id, tweet_ids):
    tweets = []
    for id in tweet_ids:
        try:
            tweet = client.get_tweet_by_id(id)
            print(tweet)
            tweets.append((int(user_id.id), 'Joe Biden', handle, int(tweet.id), str(tweet.text), tweet.created_at_datetime))
        except IndexError:
            print(f'Index Error: {tweet}')
    
    return tweets

In [31]:
import time
from math import ceil

import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint


def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [ ]:
print(client.get_user_by_screen_name('JoeBiden'))

<User id="939091">


In [39]:
def check_user_status(user_id):
    """
    True if the user is active, otherwise false (not exists or suspended).
    """
    try:
        client.get_user_by_id(user_id)
    except TwitterException as e:
        if str(e).startswith('Invalid user id'):
            return False
        raise e
    else:
        return True

In [9]:
check_user_status(1547081484695216130)

True

In [70]:
from twikit import TooManyRequests
try:
    print(tweet = client.get_tweet_by_id(939091))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

IndexError: list index out of range

In [58]:
try:
    print(client.search_tweet(
        f'from:JoeBiden since:2020-01-01 until:2021-03-01', 'Latest', count=40
    ))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

[<Tweet id="1351951465674276869">, <Tweet id="1351918910199631872">, <Tweet id="1351906918667677696">, <Tweet id="1351897267666608129">, <Tweet id="1351731172989050882">, <Tweet id="1351711040933830659">, <Tweet id="1351653131248041984">, <Tweet id="1351630258114502656">, <Tweet id="1351599720012021761">, <Tweet id="1351367275094310912">, <Tweet id="1351333542547001344">, <Tweet id="1351265605840633858">, <Tweet id="1351228360123318272">, <Tweet id="1350981483062706177">, <Tweet id="1350926118409289730">, <Tweet id="1350878051710750725">, <Tweet id="1350634446475694080">, <Tweet id="1350593782832500737">, <Tweet id="1350562220367884289">, <Tweet id="1350515133034819584">]


In [43]:
handle, user_id, tweets = get_all_tweets('JoeBiden', since='2020-01-01', until='2021-03-01')
print(handle, user_id, tweets)


20
40
60
80
100
JoeBiden <User id="939091"> ['1351951465674276869', '1351918910199631872', '1351906918667677696', '1351897267666608129', '1351731172989050882', '1351711040933830659', '1351653131248041984', '1351630258114502656', '1351599720012021761', '1351367275094310912', '1351333542547001344', '1351265605840633858', '1351228360123318272', '1350981483062706177', '1350926118409289730', '1350878051710750725', '1350634446475694080', '1350593782832500737', '1350562220367884289', '1350515133034819584', '1350480432341520384', '1350256455748882434', '1350210150481555457', '1350156296012656647', '1350077778310270978', '1349894571094810626', '1349872996920430595', '1349849776012087296', '1349563892087869442', '1349556594237792256', '1349375651883724804', '1349090628790067200', '1349068634413436929', '1348994390690246657', '1348800612742402048', '1348763039160532992', '1348636095458717696', '1348430675238678528', '1348351402624524288', '1348299813163474944', '1348064513170567168', '13479474913

In [44]:
tweets2 = process_tweets(handle, user_id, tweets)
print(tweets2)

<Tweet id="1351951465674276869">
<Tweet id="1351918910199631872">
<Tweet id="1351906918667677696">
<Tweet id="1351897267666608129">
<Tweet id="1351731172989050882">
<Tweet id="1351711040933830659">
<Tweet id="1351653131248041984">
<Tweet id="1351630258114502656">
<Tweet id="1351599720012021761">
<Tweet id="1351367275094310912">
<Tweet id="1351333542547001344">
<Tweet id="1351265605840633858">
<Tweet id="1351228360123318272">
<Tweet id="1350981483062706177">
<Tweet id="1350926118409289730">
<Tweet id="1350878051710750725">
<Tweet id="1350634446475694080">
<Tweet id="1350593782832500737">
<Tweet id="1350562220367884289">
<Tweet id="1350515133034819584">
<Tweet id="1350480432341520384">
<Tweet id="1350256455748882434">
<Tweet id="1350210150481555457">
<Tweet id="1350156296012656647">
<Tweet id="1350077778310270978">
<Tweet id="1349894571094810626">
<Tweet id="1349872996920430595">
<Tweet id="1349849776012087296">
<Tweet id="1349563892087869442">
<Tweet id="1349556594237792256">
<Tweet id=

In [45]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE tweets (
          user_id INTEGER,
          user_name TEXT,
          user_handle TEXT,
          tweet_id INTEGER,
          tweet_text TEXT,
          created_date DATETIME
          )""")

In [41]:
conn.execute('DROP TABLE tweets')

In [35]:
for tweet in tweets2:
    print(tweet)

(939091, 'Joe Biden', 'JoeBiden', 1351951465674276869, 'Now the real work begins, folks. Follow along at @POTUS as we build back better. https://t.co/fHViqSqwp2', datetime.datetime(2021, 1, 20, 17, 55, 22, tzinfo=datetime.timezone.utc))
(939091, 'Joe Biden', 'JoeBiden', 1351918910199631872, 'I love you, Jilly, and I couldn’t be more grateful to have you with me on the journey ahead. https://t.co/V4GUXAKSKg', datetime.datetime(2021, 1, 20, 15, 46, tzinfo=datetime.timezone.utc))
(939091, 'Joe Biden', 'JoeBiden', 1351906918667677696, 'Today, we begin anew. Tune in for #Inauguration2021. https://t.co/HxfU8q5riA', datetime.datetime(2021, 1, 20, 14, 58, 21, tzinfo=datetime.timezone.utc))
(939091, 'Joe Biden', 'JoeBiden', 1351897267666608129, 'It’s a new day in America.', datetime.datetime(2021, 1, 20, 14, 20, tzinfo=datetime.timezone.utc))
(939091, 'Joe Biden', 'JoeBiden', 1351731172989050882, 'Tonight, in Washington, D.C. and across the nation, we came together to honor the over 400,000 Ame

In [46]:
from sqlite3 import IntegrityError
c.executemany("INSERT INTO tweets VALUES (?,?,?,?,?,?)", tweets2)
conn.commit()


In [34]:
c.execute("SELECT * FROM tweets")
print(c.fetchall())

[(939091, 'Joe Biden', 'JoeBiden', 1351951465674276869, 'Now the real work begins, folks. Follow along at @POTUS as we build back better. https://t.co/fHViqSqwp2', '2021-01-20 17:55:22+00:00')]
